In [1]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer


model_pipeline = pipeline(
    model = "Qwen/Qwen2.5-3B-Instruct",
    # device = "cuda"
)

result = model_pipeline("What is Truvada?")
print(result)

c:\Users\munod\OneDrive\Documents\Projects\auto_surgical_report\surg_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 11.96it/s]
Device set to use cpu


[{'generated_text': 'What is Truvada? Truvada is a medication used to prevent HIV infection in people who are at high risk for becoming infected with the virus. It is taken as a daily oral dose and has been shown to reduce the risk of HIV infection by about 92% when taken consistently. Truvada is also used to treat HIV infection in people who have not previously taken antiretroviral therapy, or ART. In this case, it is typically combined with other antiretroviral drugs. Truvada is available as a tablet and must be taken exactly as prescribed by a healthcare provider. It is important to note that Truvada is only effective if taken consistently and as directed.\nIs there anything I should know about taking Truvada? Yes, there are several important things to keep in mind when taking Truvada:\n\n1. **Consistency**: Truvada must be taken every day without fail to maintain its effectiveness. Missing doses can increase the risk of HIV infection.\n\n2. **Monitoring**: Regular medical check-ups

In [2]:
print(result[0]['generated_text'])

What is Truvada? Truvada is a medication used to prevent HIV infection in people who are at high risk for becoming infected with the virus. It is taken as a daily oral dose and has been shown to reduce the risk of HIV infection by about 92% when taken consistently. Truvada is also used to treat HIV infection in people who have not previously taken antiretroviral therapy, or ART. In this case, it is typically combined with other antiretroviral drugs. Truvada is available as a tablet and must be taken exactly as prescribed by a healthcare provider. It is important to note that Truvada is only effective if taken consistently and as directed.
Is there anything I should know about taking Truvada? Yes, there are several important things to keep in mind when taking Truvada:

1. **Consistency**: Truvada must be taken every day without fail to maintain its effectiveness. Missing doses can increase the risk of HIV infection.

2. **Monitoring**: Regular medical check-ups are necessary to monitor 

In [ ]:


tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct"
)


In [30]:
train_data = load_dataset("json", data_files="training_data.json")

Generating train split: 72 examples [00:00, 9206.45 examples/s]


In [37]:
def preprocess(sample):

    sample = f"{sample["prompt"]}\n{sample["completion"]}"
    tokenized = tokenizer(
            sample,
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [38]:
data = train_data.map(preprocess)

Map: 100%|██████████| 72/72 [00:00<00:00, 641.55 examples/s]


In [42]:
print(data['train'][10])

{'prompt': 'Can rifepentine/Isoniazid be used for TB preventive therapy?', 'completion': '3 months of Rifapentine/Isoniazid (3HP) is the recommended TPT option except in pregnant and lactating women and children.', 'input_ids': [6713, 36924, 747, 26872, 38151, 930, 26975, 307, 387, 1483, 369, 30080, 70023, 15069, 5267, 18, 3951, 315, 89731, 391, 26872, 38151, 930, 26975, 307, 320, 18, 6610, 8, 374, 279, 11102, 350, 2828, 2999, 3650, 304, 20280, 323, 50544, 1095, 3198, 323, 2841, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 

In [43]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

In [44]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct", 
    device_map = "cuda", 
    torch_dtype = torch.float16
)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

AssertionError: Torch not compiled with CUDA enabled

In [45]:
from transformers import TrainingArguments, Trainer
train_args = TrainingArguments(
    num_train_epochs = 5,
    learning_rate = 0.001,
    logging_steps = 25,
    fp16 = True

)

In [ ]:
trainer = Trainer(
    args=train_args,
    model=model,
    train_dataset=data["train"]
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./guide_summary_qwen")
tokenizer.save_pretrained("./guide_summary_qwen")

In [ ]:
from transformers import pipeline

ask_summary = pipeline(
    model="./guide_summary_qwen", 
    tokenizer="./guide_summary_qwen",
    device="cuda"
)

In [ ]:
print(ask_summary("What is the preferred second line?"))[0]["generated_text"]